In [1]:
import duckdb
import pandas as pd

# Path to your DuckDB database file
duckdb_file_path = '/workspace/databases/dev.duckdb'

# Use a context manager to handle the DuckDB connection
with duckdb.connect(duckdb_file_path) as con:
    # Query all schemas
    query_schemas = "SELECT schema_name FROM information_schema.schemata"
    schemas = con.execute(query_schemas).fetchdf()
    
    # Print the DataFrame of schemas
    print("Schemas:")
    print(schemas)
    
    # Query all tables in each schema
    query_tables = """
    SELECT table_schema, table_name 
    FROM information_schema.tables 
    WHERE table_schema IN (SELECT schema_name FROM information_schema.schemata)
    """
    tables = con.execute(query_tables).fetchdf()
    
    # Print the DataFrame of tables
    print("\nTables:")
    print(tables)

    # Check if the bronze schema exists
    schema_check_query = "SELECT schema_name FROM information_schema.schemata WHERE schema_name = 'bronze'"
    schema_exists = con.execute(schema_check_query).fetchone()
    if schema_exists:
        # Query the table created by dbt
        query = "SELECT * FROM bronze.customers LIMIT 5"
        result = con.execute(query).fetchdf()

        # Check if the result is not empty and print the DataFrame
        if not result.empty:
            print(result)
        else:
            print("No records found in the customers table.")
    else:
        print("The 'bronze' schema does not exist.")

Schemas:
          schema_name
0  information_schema
1                main
2          pg_catalog
3              public
4  information_schema
5                main
6          pg_catalog
7  information_schema
8                main
9          pg_catalog

Tables:
  table_schema         table_name
0         main  test_duckdb_model
1       public  test_duckdb_model
The 'bronze' schema does not exist.


In [25]:
# Use a context manager to handle the DuckDB connection
with duckdb.connect(duckdb_file_path) as con:
    # Query to delete tables
    query_delete_bronze = "DROP SCHEMA raw CASCADE;"
    query_delete_bronze = "DROP SCHEMA bronze CASCADE;"
    query_delete_silver = "DROP SCHEMA silver CASCADE;"
    query_delete_gold = "DROP SCHEMA gold CASCADE;"
    con.execute(query_delete_bronze)
    con.execute(query_delete_silver)
    con.execute(query_delete_gold)